In [11]:
from qiskit import *
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import minimize
df = pd.read_csv (r'Data.csv')
shots = 500

In [12]:
Y = list(df['Y'])
Theta = list(df['Theta'])
State = list(df['State'])

In [21]:
Y_Train = Y[:500]
T_Train = Theta[:500]
S_Train = State[:500]

Y_Test = Y[1000:]
T_Test = Theta[1000:]
S_Test = State[1000:]

In [22]:
df_1 = pd.DataFrame({'Y':Y_Train, 'Theta':T_Train,'State':S_Train,'z': [0]*len(Y_Train)})
fig = px.scatter_3d(df_1, x='Y', y='Theta',z='z', color='State')
fig.show()

# V1

In [15]:
def Circuit_v1(Params,V):
    qc =QuantumCircuit(6,6)
    #qc.h(range(10))
    #defining
    qc.h(range(6))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    qc.ry(V[0],2)
    qc.ry(V[1],3)
    qc.cx(2,3)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),3)
    qc.cx(2,3)
    qc.ry(V[0],4)
    qc.ry(V[1],5)
    qc.cx(4,5)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),5)
    qc.cx(4,5)
    
    for i in range(int(len(Params)/12)):
        qc.ry(Params[0+12*i],0)
        qc.ry(Params[1+12*i],1)
        qc.ry(Params[2+12*i],2)
        qc.ry(Params[3+12*i],3)
        qc.ry(Params[4+12*i],2)
        qc.ry(Params[5+12*i],3)
        
        qc.crx(Params[6+12*i],0,1)
        qc.crx(Params[7+12*i],1,2)
        qc.crx(Params[8+12*i],2,3)
        qc.crx(Params[9+12*i],3,4)
        qc.crx(Params[10+12*i],4,5)
        qc.crx(Params[11+12*i],5,0)
    
    qc.measure([0,1,2,3,4,5],[0,1,2,3,4,5])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    return(counts)
    
    
    
    
    

In [16]:
list(Circuit_v1([0]*12,[0,0]).keys())[1]

'011001'

In [17]:
output = Circuit_v1([0]*12,[0,0])

In [18]:
output

{'011001': 47,
 '011010': 41,
 '000001': 1,
 '110110': 3,
 '101010': 40,
 '100001': 2,
 '001010': 9,
 '011000': 4,
 '101110': 7,
 '000101': 7,
 '110010': 1,
 '010110': 56,
 '100100': 5,
 '000010': 1,
 '101011': 8,
 '110101': 4,
 '001110': 1,
 '100101': 45,
 '101101': 2,
 '111010': 1,
 '100110': 40,
 '101001': 44,
 '010101': 37,
 '010001': 4,
 '101000': 7,
 '001011': 1,
 '100010': 7,
 '111001': 5,
 '010100': 7,
 '011011': 4,
 '001001': 4,
 '011101': 9,
 '100111': 7,
 '000111': 1,
 '101111': 1,
 '000110': 6,
 '011110': 10,
 '101100': 3,
 '010000': 2,
 '010010': 5,
 '111000': 1,
 '010111': 8,
 '011111': 1,
 '100011': 1}

In [25]:
2**6/4

16.0

In [29]:
type(output[list(output.keys())[1]])

int

In [26]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        output = Circuit_v1(Params,[Y_Train[i],T_Train[i]])
        keys = list(output.keys())
        result = [0]*4
        for j in range(len(output)):
            if int(keys[j],2)<16:
                result[0] += output[keys[j]]
            elif 15<int(keys[j],2)<32:
                result[1] += output[keys[j]]
            elif 31<int(keys[j],2)<48:
                result[2] += output[keys[j]]
            elif 47<int(keys[j],2):
                result[3] += output[keys[j]]
                
        actual = [0]*4
        actual[S_Train[i]] = shots
        cost += sum((np.array(result)-np.array(actual))**2)
    return(cost)

In [27]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v1(Params,[Y_Test[i],T_Test[i]])
        keys = list(output.keys())
        result = [0]*4
        for j in range(len(output)):
            if int(keys[j],2)<16:
                result[0] += output[keys[j]]
            elif 15<int(keys[j],2)<32:
                result[1] += output[keys[j]]
            elif 31<int(keys[j],2)<48:
                result[2] += output[keys[j]]
            elif 47<int(keys[j],2):
                result[3] += output[keys[j]]
        if result.index(max(result))== S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [28]:
# Params = np.random.uniform(low=-2.6, high=2.6, size=(8,))
Params = [0]*16

In [29]:
print(Cost(Params))
print(eff(Params))

158724146
0.10675


In [30]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)

In [31]:
minimum.x

array([ 0.08177536,  0.91733349,  0.71530753,  0.39002225, -0.30034717,
       -0.87526267,  1.18713211, -0.18900129, -1.16650398,  2.01397984,
        3.67300295, -0.1753296 ,  1.87631291,  0.14150281,  0.11008806,
       -0.57190999])

In [32]:
print(Cost(minimum.x))
print(eff(minimum.x))

78959874
0.48675


# 24 param

In [34]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(24,))

In [35]:
print(Cost(Params))
print(eff(Params))

126136242
0.13675


In [36]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)
minimum

     fun: 79841752.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 195
  status: 1
 success: True
       x: array([-0.19189892,  0.58897692, -0.86101898,  1.30255212,  0.79508   ,
       -1.66228557,  0.02398332,  2.05087466,  1.79106957,  2.61626854,
       -0.37803839,  2.76879768, -1.86095734,  1.83923345,  2.1284851 ,
       -0.56302968, -2.25308522,  1.97633228,  1.83927488, -2.49925268,
        0.41163122, -1.99636507,  3.34959936,  2.14909829])

In [37]:
print(Cost(minimum.x))
print(eff(minimum.x))

79626254
0.45275


# 36 params

In [39]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(36,))

In [40]:
print(Cost(Params))
print(eff(Params))

99723140
0.17675


In [41]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)
minimum

     fun: 72942678.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 305
  status: 1
 success: True
       x: array([ 2.39272969, -0.85500707,  1.22077794, -0.76602996,  0.84423365,
       -0.67098766,  1.64349845, -1.90469393,  1.25306075,  1.90232592,
       -2.54207708, -0.35067321, -1.06054438, -2.45710741,  0.7561049 ,
        2.45758437, -1.84607365,  0.98947226,  2.4868851 ,  3.23148875,
       -0.57102817, -2.02379257, -0.9271198 , -2.34836855, -1.78966306,
        0.62807234,  0.41799448,  0.08648372, -2.61064526, -0.10990982,
       -0.20042816,  1.66031451, -0.95921163, -3.2231019 , -0.82972636,
       -0.63258818])

In [42]:
print(Cost(minimum.x))
print(eff(minimum.x))

72802862
0.55425
